In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Ensure project root is on path
PROJECT_ROOT = Path.cwd().resolve().parents[0]
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from project_code.functions import *

torch.set_num_threads(8)
torch.set_num_interop_threads(1)

In [2]:
np.random.seed(42)
torch.manual_seed(42)

In [3]:
# Download PBP
seasons = range(2011,2026)
print("Loading play-by-play for seasons:", seasons)
raw_pbp = nfl.import_pbp_data(seasons, downcast=False)  # returns a DataFrame (likely large)

print("Rows loaded:", raw_pbp.shape[0])
raw_pbp.head()

Loading play-by-play for seasons: range(2011, 2026)
2011 done.
2012 done.
2013 done.
2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
2024 done.
2025 done.
Rows loaded: 722912


,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,was_pressure,route,defense_man_zone_type,defense_coverage_type,offense_names,defense_names,offense_positions,defense_positions,offense_numbers,defense_numbers
0,1.0,2011_01_ATL_CHI,2011091105,CHI,ATL,REG,1,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36.0,2011_01_ATL_CHI,2011091105,CHI,ATL,REG,1,CHI,home,ATL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,69.0,2011_01_ATL_CHI,2011091105,CHI,ATL,REG,1,CHI,home,ATL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,91.0,2011_01_ATL_CHI,2011091105,CHI,ATL,REG,1,CHI,home,ATL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,112.0,2011_01_ATL_CHI,2011091105,CHI,ATL,REG,1,CHI,home,ATL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
cols_to_keep = ['play_type', 'season', 'home_wp_post', 'away_wp_post', 'weather', 'yardline_100', 'ydstogo',
               'game_seconds_remaining', 'half_seconds_remaining', 'posteam', 'defteam',
               'posteam_timeouts_remaining', 'defteam_timeouts_remaining', 'kick_distance', 'touchback',
                'return_yards', 'first_down', 'touchdown', 'game_id', 'score_differential',
                'home_team', 'away_team', 'home_score', 'away_score', 'down', 'field_goal_result', 'penalty',
               'home_coach', 'away_coach', 'spread_line', 'total_line']

pbp = raw_pbp.loc[:, cols_to_keep].copy()

In [5]:
action_to_col = {
    "punt": "punt",
    "field_goal": "field_goal",
    "run": "go",
    "pass": "go"
}

pbp["play_type_actual"] = pbp["play_type"].map(action_to_col)
pbp = pbp[pbp.play_type_actual.isin(['punt', 'go', 'field_goal'])]
pbp = deconstruct_weather(pbp)
pbp = pbp[pbp.penalty == 0]
pbp['fg_made'] = (pbp["field_goal_result"] == "made").astype(int)

action_to_ewp_col = {
    "punt": "ewp_punt",
    "field_goal": "ewp_fg",
    "go": "ewp_go"
}
pbp["actual_ewp_col"] = pbp["play_type_actual"].map(action_to_ewp_col)

pbp["possession_coach"] = np.where(pbp["posteam"] == pbp["home_team"], pbp["home_coach"], pbp["away_coach"])
pbp["defending_coach"] = np.where(pbp["posteam"] == pbp["home_team"], pbp["away_coach"], pbp["home_coach"])
pbp["possession_spread_line"] = np.where(pbp["posteam"] == pbp["home_team"], pbp["spread_line"], -pbp["spread_line"])

pbp = create_features(pbp)

In [6]:
seasons = pbp.season.unique() # seasons
test_season = seasons.max()

pbp_train = pbp[pbp.season != test_season]
pbp_test = pbp[pbp.season == test_season]

In [8]:
# --- Drop rows missing home/away WP
wp_df = pbp_train.dropna(subset=["home_wp_post", "away_wp_post"]).copy()

# --- Define features
wp_features = [
    "yardline_100",
    "down",
    "ydstogo",
    "game_seconds_remaining",
    "half_seconds_remaining",
    "score_differential",
    "posteam_timeouts_remaining",
    "defteam_timeouts_remaining",
    "temp_F",
    "wind_mph",
    "possession_spread_line",
    "total_line",
    "is_redzone",
    "is_goal_to_go",
    "log_ydstogo",
    "log_game_seconds_remaining",
    "abs_score_differential",
    "score_time_ratio",
]

engineered = {
    "is_redzone",
    "is_goal_to_go",
    "log_ydstogo",
    "log_game_seconds_remaining",
    "abs_score_differential",
    "score_time_ratio",
}

wp_base_features = [f for f in wp_features if f not in engineered]

# --- Define posteam WP target
wp_df["wp_target"] = np.where(
    wp_df["posteam"] == wp_df["home_team"],
    wp_df["home_wp_post"],
    wp_df["away_wp_post"],
)

# NEW: create engineered columns on the full frame
wp_df = create_features(wp_df)

wp_df = wp_df.reset_index(drop=True)

X_wp = wp_df[wp_features]
y_wp = wp_df["wp_target"]

epsilon = 1e-6
y_wp_clipped = y_wp.clip(epsilon, 1 - epsilon).reset_index(drop=True)

monotone_constraints_dict = {
    "yardline_100": -1,
    "ydstogo": -1,
    "score_differential": 1,
    "possession_spread_line": -1,
}

mono_tuple = tuple(monotone_constraints_dict.get(c, 0) for c in X_wp.columns)

wp_folds = make_temporal_folds(wp_df)

In [9]:
wp_storage = "sqlite:///" + str((Path.cwd().parent / "optuna" / "wp_study.db").resolve())

wp_study = optuna.create_study(
    study_name="wp_study_v9",
    direction="minimize",
    storage=wp_storage,
    load_if_exists=True
)

wp_fixed_params = {
    "n_estimators": 2000,
    "eval_metric": "rmse",
    "tree_method": "hist",
    "early_stopping_rounds": 100,
    "max_bin": 128,
    "verbosity": 0,
    "n_jobs": 14,
}

def objective(trial):
    
    return wp_objective(
        trial,
        wp_fixed_params=wp_fixed_params,
        X_wp=X_wp,
        y_wp_clipped=y_wp_clipped,
        wp_folds=wp_folds,
        mono_tuple=mono_tuple,
    )

wp_study.optimize(objective, n_trials=50, n_jobs=1)

[I 2026-01-22 13:49:45,847] A new study created in RDB with name: wp_study_v9
[I 2026-01-22 13:52:15,603] Trial 0 finished with value: 0.05860858038067818 and parameters: {'learning_rate': 0.057695122873117004, 'max_depth': 4, 'subsample': 0.7817071129379796, 'colsample_bytree': 0.8682218518226015, 'min_child_weight': 6.665416554243339, 'reg_lambda': 0.07591078877312794}. Best is trial 0 with value: 0.05860858038067818.
[I 2026-01-22 13:55:53,291] Trial 1 finished with value: 0.058786116540431976 and parameters: {'learning_rate': 0.035611984048418505, 'max_depth': 4, 'subsample': 0.8405297890922699, 'colsample_bytree': 0.7248719664440452, 'min_child_weight': 5.664971727839539, 'reg_lambda': 0.6317744141137664}. Best is trial 0 with value: 0.05860858038067818.
[I 2026-01-22 14:31:36,292] Trial 2 finished with value: 0.05864125117659569 and parameters: {'learning_rate': 0.02691964211766871, 'max_depth': 5, 'subsample': 0.8922857227597861, 'colsample_bytree': 0.7269771808324474, 'min_chil

[I 2026-01-22 16:29:32,070] Trial 23 finished with value: 0.05838024988770485 and parameters: {'learning_rate': 0.05869461711588678, 'max_depth': 5, 'subsample': 0.7648011331768786, 'colsample_bytree': 0.810129256328011, 'min_child_weight': 39.079271850901904, 'reg_lambda': 20.855970870820656}. Best is trial 21 with value: 0.05835916846990585.
[I 2026-01-22 16:32:37,107] Trial 24 finished with value: 0.05842825025320053 and parameters: {'learning_rate': 0.040662104171738434, 'max_depth': 5, 'subsample': 0.7950199925942895, 'colsample_bytree': 0.838601034352931, 'min_child_weight': 96.98539981491943, 'reg_lambda': 2.4855264047566097}. Best is trial 21 with value: 0.05835916846990585.
[I 2026-01-22 16:35:08,125] Trial 25 finished with value: 0.05837671458721161 and parameters: {'learning_rate': 0.05661135383204965, 'max_depth': 5, 'subsample': 0.8181655601174088, 'colsample_bytree': 0.7862267175720046, 'min_child_weight': 10.670462162867278, 'reg_lambda': 10.541167006064093}. Best is tri

[I 2026-01-22 17:32:10,563] Trial 46 finished with value: 0.05847523361444473 and parameters: {'learning_rate': 0.07929192705864811, 'max_depth': 5, 'subsample': 0.8423773628102887, 'colsample_bytree': 0.7645824856039, 'min_child_weight': 79.28024790654486, 'reg_lambda': 49.61390769794947}. Best is trial 21 with value: 0.05835916846990585.
[I 2026-01-22 17:34:07,249] Trial 47 finished with value: 0.0583813413977623 and parameters: {'learning_rate': 0.060511449532133985, 'max_depth': 5, 'subsample': 0.7454503676858342, 'colsample_bytree': 0.8239858188127608, 'min_child_weight': 18.872918346223916, 'reg_lambda': 7.359546447988118}. Best is trial 21 with value: 0.05835916846990585.
[I 2026-01-22 17:36:09,955] Trial 48 finished with value: 0.05837496370077133 and parameters: {'learning_rate': 0.05538001525631526, 'max_depth': 5, 'subsample': 0.8131444378737875, 'colsample_bytree': 0.8470933987966186, 'min_child_weight': 8.482794217617032, 'reg_lambda': 16.513718958405526}. Best is trial 21

In [10]:
wp_best_params = wp_study.best_params.copy()
wp_best_score = wp_study.best_value

print("Best CV RMSE:", wp_best_score)
print()
print("Best params:", wp_best_params)

# Remove any tuned keys that collide with fixed keys (robust to old studies)
for k in list(wp_best_params.keys()):
    if k in wp_fixed_params:
        wp_best_params.pop(k)

final_params = {**wp_fixed_params, **wp_best_params, "monotone_constraints": mono_tuple}

# EARLY STOPPING REQUIRES eval_set -> remove for full-data fit
final_params.pop("early_stopping_rounds", None)

wp_model = XGBRegressor(**final_params)
wp_model.fit(X_wp, y_wp_clipped)

Best CV RMSE: 0.05835916846990585

Best params: {'colsample_bytree': 0.7828475262099281, 'learning_rate': 0.0512377189688351, 'max_depth': 5, 'min_child_weight': 27.021545335021447, 'reg_lambda': 44.96430628444343, 'subsample': 0.8063590856939649}


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7828475262099281, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='rmse', feature_types=None, gamma=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.0512377189688351,
             max_bin=128, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=27.021545335021447, missing=nan,
             monotone_constraints=(-1, -1, -1, 0, 0, 1, 1, -1, 0, 0, -1, 0, 0,
                                   0, 0, 0, 0, 0),
             multi_strategy=None, n_estimators=2000, n_jobs=14,
             num_parallel_tree=None, random_state=None, ...)

In [11]:
wp_importances = (pd.DataFrame({
        "feature": wp_features,
        "importance": wp_model.feature_importances_
    })
    .sort_values("importance", ascending=False)
)

print(wp_importances.to_string(index=False))

                   feature  importance
        score_differential    0.910531
          score_time_ratio    0.031835
              yardline_100    0.017698
                is_redzone    0.015319
                      down    0.007383
posteam_timeouts_remaining    0.004965
defteam_timeouts_remaining    0.004223
log_game_seconds_remaining    0.001752
                   ydstogo    0.001584
    game_seconds_remaining    0.001477
    abs_score_differential    0.001125
             is_goal_to_go    0.000673
               log_ydstogo    0.000384
    half_seconds_remaining    0.000373
    possession_spread_line    0.000218
                total_line    0.000161
                  wind_mph    0.000149
                    temp_F    0.000148


In [25]:
# Create punt_df with only punt plays
punt_df = pbp_train[pbp_train.play_type_actual == "punt"].dropna(subset=["kick_distance", "return_yards"]).copy()

punt_df["net_punt"] = punt_df["kick_distance"] - punt_df["return_yards"]
punt_df.loc[punt_df["touchback"] == 1, "net_punt"] = punt_df["yardline_100"] - 20

punt_df = punt_df.reset_index(drop=True)

punt_folds = make_temporal_folds(punt_df)

punt_features = [
    "yardline_100", 
    "game_seconds_remaining", 
    "half_seconds_remaining",
    "score_differential",
    "posteam_timeouts_remaining",
    "defteam_timeouts_remaining",
    "temp_F",
    "wind_mph",
    "possession_spread_line",
    "total_line",
    "is_redzone",
    "is_goal_to_go",
    "log_ydstogo",
    "log_game_seconds_remaining",
    "abs_score_differential",
    "score_time_ratio"
]

X_punt = punt_df[punt_features].to_numpy(dtype=np.float32, copy=False)
y_punt = punt_df["net_punt"].to_numpy(dtype=np.float32, copy=False)

In [28]:
punt_storage = "sqlite:///" + str((Path.cwd().parent / "optuna" / "punt_study.db").resolve())

punt_study = optuna.create_study(
    study_name="punt_study_xgb_v1",
    direction="minimize",
    storage=punt_storage,
    load_if_exists=True
)

def objective(trial):
    return punt_objective(
        trial,
        X_punt=X_punt,
        y_punt=y_punt,
        punt_folds=punt_folds
    )

punt_study.optimize(objective, n_trials=50, n_jobs=1)

[I 2026-01-24 10:43:13,034] Using an existing study with name 'punt_study_xgb_v1' instead of creating a new one.
[I 2026-01-24 10:43:15,553] Trial 3 finished with value: 10.046379089355469 and parameters: {'max_depth': 4, 'learning_rate': 0.03588088233061935, 'subsample': 0.8215718231781188, 'colsample_bytree': 0.658701017931485, 'min_child_weight': 48.47984330513557, 'reg_lambda': 0.023194429183651364, 'reg_alpha': 0.3331898482212518, 'gamma': 4.734877168807344}. Best is trial 0 with value: 10.0402250289917.
[I 2026-01-24 10:43:18,103] Trial 4 finished with value: 10.040709495544434 and parameters: {'max_depth': 3, 'learning_rate': 0.032475038617994716, 'subsample': 0.9113662745650011, 'colsample_bytree': 0.7255295065000168, 'min_child_weight': 13.726580478082257, 'reg_lambda': 0.04775731236662879, 'reg_alpha': 0.43049668140228614, 'gamma': 0.10083664313539742}. Best is trial 0 with value: 10.0402250289917.
[I 2026-01-24 10:43:20,245] Trial 5 finished with value: 10.067732810974121 an

[I 2026-01-24 10:44:16,140] Trial 23 finished with value: 10.037801742553711 and parameters: {'max_depth': 3, 'learning_rate': 0.01644857230531302, 'subsample': 0.8110244316605628, 'colsample_bytree': 0.6892956493727658, 'min_child_weight': 18.65429019108618, 'reg_lambda': 9.706258892470656, 'reg_alpha': 6.467000237633041e-06, 'gamma': 2.934825794948744}. Best is trial 23 with value: 10.037801742553711.
[I 2026-01-24 10:44:21,213] Trial 24 finished with value: 10.037665367126465 and parameters: {'max_depth': 3, 'learning_rate': 0.016256434924570268, 'subsample': 0.8049735487810779, 'colsample_bytree': 0.6954418734695528, 'min_child_weight': 25.435152499879393, 'reg_lambda': 11.733824682414363, 'reg_alpha': 6.948262018209163e-06, 'gamma': 4.112438800740632}. Best is trial 24 with value: 10.037665367126465.
[I 2026-01-24 10:44:28,230] Trial 25 finished with value: 10.044268608093262 and parameters: {'max_depth': 4, 'learning_rate': 0.010096145107468788, 'subsample': 0.7580154211083752, '

[I 2026-01-24 10:45:54,398] Trial 43 finished with value: 10.03847885131836 and parameters: {'max_depth': 3, 'learning_rate': 0.01202278259178739, 'subsample': 0.6365414125674855, 'colsample_bytree': 0.7377361827819117, 'min_child_weight': 32.236542613689835, 'reg_lambda': 6.267764987416046, 'reg_alpha': 0.00012329717481000978, 'gamma': 3.649562214773029}. Best is trial 37 with value: 10.036905288696289.
[I 2026-01-24 10:46:00,224] Trial 44 finished with value: 10.038505554199219 and parameters: {'max_depth': 3, 'learning_rate': 0.010973727931368719, 'subsample': 0.6684392240348752, 'colsample_bytree': 0.7158066903276065, 'min_child_weight': 20.85145088217019, 'reg_lambda': 20.54902784619834, 'reg_alpha': 1.3885015170753486e-05, 'gamma': 3.2127156377729396}. Best is trial 37 with value: 10.036905288696289.
[I 2026-01-24 10:46:03,884] Trial 45 finished with value: 10.038921356201172 and parameters: {'max_depth': 3, 'learning_rate': 0.01990695416888555, 'subsample': 0.6453639776091472, '

In [29]:
punt_best_params = punt_study.best_params.copy()
punt_best_score = punt_study.best_value

print("Best CV RMSE (yards):", punt_best_score)
print()
print("Best params:", punt_best_params)

# ---- Train final model on full data using best params ----
punt_fixed_params = {
    "n_estimators": 5000,
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "tree_method": "hist",
    "max_bin": 256,
    "verbosity": 0,
    "n_jobs": 14,
}

punt_model = XGBRegressor(**punt_fixed_params, **punt_best_params)
punt_model.fit(X_punt, y_punt, verbose=False)

Best CV RMSE (yards): 10.036905288696289

Best params: {'colsample_bytree': 0.6675915033387726, 'gamma': 3.780171164232878, 'learning_rate': 0.014418472304592157, 'max_depth': 3, 'min_child_weight': 17.024584871340977, 'reg_alpha': 4.436512591858259e-05, 'reg_lambda': 5.499329449631277, 'subsample': 0.6758355547764563}


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6675915033387726, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='rmse', feature_types=None, gamma=3.780171164232878,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.014418472304592157,
             max_bin=256, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=17.024584871340977, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=5000,
             n_jobs=14, num_parallel_tree=None, random_state=None, ...)

In [33]:
punt_importances = (pd.DataFrame({
        "feature": punt_features,
        "importance": punt_model.feature_importances_
    })
    .sort_values("importance", ascending=False)
)

print(punt_importances.to_string(index=False))

                   feature  importance
              yardline_100    0.341603
               log_ydstogo    0.058043
                    temp_F    0.055400
defteam_timeouts_remaining    0.053610
log_game_seconds_remaining    0.051258
          score_time_ratio    0.050589
    half_seconds_remaining    0.050156
    game_seconds_remaining    0.049993
        score_differential    0.049810
    possession_spread_line    0.049588
posteam_timeouts_remaining    0.049533
                total_line    0.047905
                  wind_mph    0.046366
    abs_score_differential    0.046146
                is_redzone    0.000000
             is_goal_to_go    0.000000


In [17]:
# --- Filter to field goal attempts only ---
fg_df = pbp_train[pbp_train.play_type_actual == "field_goal"].dropna(subset=["field_goal_result"]).copy()
fg_df = fg_df[fg_df.field_goal_result.isin(['made', 'missed', 'blocked'])]

# Field goal
fg_features = [
    "yardline_100",
    "game_seconds_remaining",
    "half_seconds_remaining",
    "score_differential",
    "temp_F",
    "wind_mph",
    "possession_spread_line",
    "total_line",
    "is_redzone",
    "is_goal_to_go",
    "log_ydstogo",
    "log_game_seconds_remaining",
    "abs_score_differential",
    "score_time_ratio"
]

X_fg = fg_df[fg_features]
y_fg = fg_df["fg_made"]

fg_folds = make_temporal_folds(fg_df)

fg_oof_pred = pd.Series(index=fg_df.index, dtype=float)

for fold_num, (train_idx, val_idx) in enumerate(fg_folds, 1):
    X_train = X_fg.loc[train_idx]
    y_train = y_fg.loc[train_idx]
    X_val   = X_fg.loc[val_idx]

    fg_model_lr_fold = make_pipeline(
        StandardScaler(),
        LogisticRegression(
            solver="lbfgs",
            max_iter=5000,
            C=1.0,
        )
    )

    fg_model_lr_fold.fit(X_train, y_train)
    fg_oof_pred.loc[val_idx] = fg_model_lr_fold.predict_proba(X_val)[:, 1]

mask = fg_oof_pred.notna()
fg_oof_rmse = np.sqrt(np.mean((fg_oof_pred[mask] - y_fg[mask]) ** 2))
print("FG (LogReg) OOF RMSE:", fg_oof_rmse)

# final model on all data (scaled consistently)
fg_model = make_pipeline(
    StandardScaler(),
    LogisticRegression(solver="lbfgs", max_iter=5000, C=1.0)
)
fg_model.fit(X_fg, y_fg)

FG (LogReg) OOF RMSE: 0.338991485810632


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(max_iter=5000))])

In [18]:
# pull fitted LR out of the pipeline
lr_ns = fg_model.named_steps["logisticregression"]

coef = lr_ns.coef_.ravel()  # shape (n_features,)
importance = np.abs(coef)

imp_df = pd.DataFrame({
    "feature": X_fg.columns,
    "coef_z": coef,                # per 1 std dev increase in feature
    "abs_coef_z": importance,
}).sort_values("abs_coef_z", ascending=False)

imp_df

,feature,coef_z,abs_coef_z
0,yardline_100,-0.970823,0.970823
9,is_goal_to_go,0.123009,0.123009
5,wind_mph,-0.092400,0.092400
8,is_redzone,0.078099,0.078099
1,game_seconds_remaining,-0.069404,0.069404
11,log_game_seconds_remaining,0.062407,0.062407
2,half_seconds_remaining,0.061242,0.061242
4,temp_F,0.058142,0.058142
10,log_ydstogo,-0.028218,0.028218
7,total_line,0.014803,0.014803


In [19]:
# Filter to 4th-down go-for-it plays
go_df = pbp_train[
    (pbp_train['down'] == 4) &
    (pbp_train['play_type_actual'] == 'go')  # filters out punts/FGs
].copy()

# Target: did the team convert?
go_df = go_df.dropna(subset=['first_down'])

# Go-for-it conversion
go_df["success"] = (
    (go_df["first_down"] == 1) |
    (go_df["touchdown"] == 1)
).astype(int)

# Reset index to avoid any issues
go_df = go_df.reset_index(drop=True)

# Make temporal folds based on seasons in punt_df
go_folds = make_temporal_folds(go_df)

# Features to predict net punt
go_features = [
    "yardline_100",
    "ydstogo",
    "game_seconds_remaining",
    "half_seconds_remaining",
    "score_differential",
    "posteam_timeouts_remaining",
    "defteam_timeouts_remaining",
    "temp_F",
    "wind_mph",
    "possession_spread_line",
    "total_line",
    "is_redzone",
    "is_goal_to_go",
    "log_ydstogo",
    "log_game_seconds_remaining",
    "abs_score_differential",
    "score_time_ratio"
]

X_go = go_df[go_features].values
y_go = go_df["success"].values

monotone_constraints = [
    -1,  # yardline_100 (farther → worse)
    -1,  # ydstogo (longer → worse)
    0,   # game_seconds_remaining
    0,   # half_seconds_remaining
    0,   # score_differential
    0,   # posteam_timeouts_remaining
    0,   # defteam_timeouts_remaining
    0,   # temp_F
    0,   # wind_mph
    0,   # possession_spread_line
    0,   # total_line
    0,   # is_redzone
    0,   # is_goal_to_go
    -1,  # log_ydstogo
    0,   # log_game_seconds_remaining
    0,   # abs_score_differential
    0    # score_time_ratio
]

mono_tuple_go = tuple(monotone_constraints)

In [20]:
go_storage = "sqlite:///" + str((Path.cwd().parent / "optuna" / "go_study.db").resolve())

sampler = optuna.samplers.TPESampler(seed=42, n_startup_trials=10)

go_study = optuna.create_study(
    study_name="go_study_v5",
    direction="minimize",
    storage=go_storage,
    load_if_exists=True,
    sampler=sampler
)

go_fixed_params = {
    "n_estimators": 1000,
    "eval_metric": "logloss",
    "tree_method": "hist",
    "early_stopping_rounds": 50,
    "max_bin": 128,
    "verbosity": 0,
    "n_jobs": 14,
    "objective": "binary:logistic"
}

def objective(trial):
    
    return go_objective(
        trial,
        go_fixed_params=go_fixed_params,
        X_go=X_go,
        y_go=y_go,
        go_folds=go_folds,
        mono_tuple_go=mono_tuple_go
    )

go_study.optimize(objective, n_trials=50, n_jobs=1)

[I 2026-01-22 18:06:35,154] A new study created in RDB with name: go_study_v5
[I 2026-01-22 18:06:42,661] Trial 0 finished with value: 0.6451785422787121 and parameters: {'max_depth': 3, 'learning_rate': 0.07471658997996457, 'subsample': 0.846398788362281, 'colsample_bytree': 0.8197316968394073, 'min_child_weight': 1.841072920573868, 'reg_lambda': 0.037758875456826865, 'reg_alpha': 2.231010801867921e-06, 'gamma': 1.7323522915498704}. Best is trial 0 with value: 0.6451785422787121.
[I 2026-01-22 18:06:51,285] Trial 1 finished with value: 0.6442722861046947 and parameters: {'max_depth': 3, 'learning_rate': 0.05337429672856024, 'subsample': 0.7041168988591604, 'colsample_bytree': 0.8939819704323989, 'min_child_weight': 25.95942550311264, 'reg_lambda': 0.06101491367302711, 'reg_alpha': 1.2329623163659841e-05, 'gamma': 0.36680901970686763}. Best is trial 1 with value: 0.6442722861046947.
[I 2026-01-22 18:06:59,381] Trial 2 finished with value: 0.6442053165767722 and parameters: {'max_depth'

[I 2026-01-22 18:10:23,437] Trial 20 finished with value: 0.6444241918215835 and parameters: {'max_depth': 3, 'learning_rate': 0.06336779532846101, 'subsample': 0.7888213941899516, 'colsample_bytree': 0.834669340930887, 'min_child_weight': 9.386093385208067, 'reg_lambda': 5.224267821605316, 'reg_alpha': 1.2120629279159232e-05, 'gamma': 0.19722383057835446}. Best is trial 9 with value: 0.6438261091579496.
[I 2026-01-22 18:10:30,410] Trial 21 finished with value: 0.6441648793579634 and parameters: {'max_depth': 2, 'learning_rate': 0.06664303057983002, 'subsample': 0.7645474471447712, 'colsample_bytree': 0.7845935896576969, 'min_child_weight': 4.447673571243616, 'reg_lambda': 8.239009438733424, 'reg_alpha': 5.586765104569471e-06, 'gamma': 0.56493203319448}. Best is trial 9 with value: 0.6438261091579496.
[I 2026-01-22 18:10:35,852] Trial 22 finished with value: 0.6445255560709318 and parameters: {'max_depth': 2, 'learning_rate': 0.0788334686062176, 'subsample': 0.7254287684017745, 'colsam

[I 2026-01-22 18:13:01,833] Trial 40 finished with value: 0.645001539178377 and parameters: {'max_depth': 3, 'learning_rate': 0.0438500388640632, 'subsample': 0.883042484655321, 'colsample_bytree': 0.8299738856199949, 'min_child_weight': 16.666244654070944, 'reg_lambda': 0.1746189076315655, 'reg_alpha': 0.0021814635053105606, 'gamma': 0.2923926752167989}. Best is trial 25 with value: 0.643642096303433.
[I 2026-01-22 18:13:09,593] Trial 41 finished with value: 0.6440971745246377 and parameters: {'max_depth': 2, 'learning_rate': 0.05838236284862739, 'subsample': 0.8002419294760929, 'colsample_bytree': 0.8115473334120387, 'min_child_weight': 18.269118800266693, 'reg_lambda': 0.028546067034378057, 'reg_alpha': 0.004605386795421558, 'gamma': 0.8995785011939417}. Best is trial 25 with value: 0.643642096303433.
[I 2026-01-22 18:13:17,556] Trial 42 finished with value: 0.6439308813877669 and parameters: {'max_depth': 2, 'learning_rate': 0.05583169874358669, 'subsample': 0.8140499213634084, 'co

In [21]:
go_best_params = go_study.best_params.copy()
go_best_score  = go_study.best_value

print("Best CV logloss:", go_best_score)
print()
print("Best params:", go_best_params)

# Remove any tuned keys that collide with fixed keys (robust to old studies)
for k in list(go_best_params.keys()):
    if k in go_fixed_params:
        go_best_params.pop(k)

final_params = {
    **go_fixed_params,
    **go_best_params,
    "monotone_constraints": mono_tuple_go,
}

# EARLY STOPPING REQUIRES eval_set -> remove for full-data fit
final_params.pop("early_stopping_rounds", None)

go_model = XGBClassifier(**final_params)
go_model.fit(X_go, y_go)

Best CV logloss: 0.6434090971335941

Best params: {'colsample_bytree': 0.7391057725692328, 'gamma': 0.4776848193389836, 'learning_rate': 0.07497759476499909, 'max_depth': 2, 'min_child_weight': 35.03578414252553, 'reg_alpha': 0.16813121525170766, 'reg_lambda': 0.160391049919331, 'subsample': 0.7503806706428661}


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7391057725692328, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None,
              gamma=0.4776848193389836, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.07497759476499909,
              max_bin=128, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=35.03578414252553, missing=nan,
              monotone_constraints=(-1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1,
                                    0, 0, 0),
              multi_strategy=None, n_estimators=1000, n_jobs=14,
              num_parallel_tree=None, random_state=None, ...)

In [22]:
go_importances = (pd.DataFrame({
        "feature": go_features,
        "importance": go_model.feature_importances_
    })
    .sort_values("importance", ascending=False)
)

print(go_importances.to_string(index=False))

                   feature  importance
                   ydstogo    0.314600
               log_ydstogo    0.229104
             is_goal_to_go    0.055766
                is_redzone    0.040555
defteam_timeouts_remaining    0.032828
    half_seconds_remaining    0.032344
                total_line    0.031280
posteam_timeouts_remaining    0.028669
          score_time_ratio    0.028666
              yardline_100    0.027467
                  wind_mph    0.027152
    abs_score_differential    0.026796
    possession_spread_line    0.026738
                    temp_F    0.025778
        score_differential    0.024920
    game_seconds_remaining    0.024316
log_game_seconds_remaining    0.023019


In [31]:
pbp_pre_train, pbp_fourth_train = create_df_with_ewp(
    pbp_train,
    wp_model=wp_model,
    go_model=go_model,
    fg_model=fg_model,
    punt_model=punt_model,
    wp_features=wp_features,
    wp_base_features=wp_base_features,
    go_features=go_features,
    fg_features=fg_features,
    punt_features=punt_features
)

pbp_pre_test, pbp_fourth_test = create_df_with_ewp(
    pbp_test,
    wp_model=wp_model,
    go_model=go_model,
    fg_model=fg_model,
    punt_model=punt_model,
    wp_features=wp_features,
    wp_base_features=wp_base_features,
    go_features=go_features,
    fg_features=fg_features,
    punt_features=punt_features
)

In [34]:
# testing git workflow

timestamp = datetime.now().strftime("%Y%m%d_%H%M")

# Variables
joblib.dump(test_season, f"../exports/test_season_{timestamp}.joblib")
joblib.dump(wp_features, f"../exports/wp_features_{timestamp}.joblib")
joblib.dump(wp_base_features, f"../exports/wp_base_features_{timestamp}.joblib")
joblib.dump(go_features, f"../exports/go_features_{timestamp}.joblib")
joblib.dump(fg_features, f"../exports/fg_features_{timestamp}.joblib")
joblib.dump(punt_features, f"../exports/punt_features_{timestamp}.joblib")
joblib.dump(go_folds, f"../exports/go_folds_{timestamp}.joblib")

# Dataframes
raw_pbp.to_parquet(f"../exports/raw_pbp{timestamp}.parquet")
pbp.to_parquet(f"../exports/pbp{timestamp}.parquet")
pbp_fourth_train.to_parquet(f"../exports/pbp_fourth_train_{timestamp}.parquet")
pbp_fourth_test.to_parquet(f"../exports/pbp_fourth_test_{timestamp}.parquet")

# Models
joblib.dump(wp_model, f"../exports/wp_model_{timestamp}.joblib")
joblib.dump(go_model, f"../exports/go_model_{timestamp}.joblib")
joblib.dump(fg_model, f"../exports/fg_model_{timestamp}.joblib")
joblib.dump(punt_model, f"../exports/punt_model_{timestamp}.joblib")

# Objects
joblib.dump(X_scaler, f"../exports/X_scaler_{timestamp}.joblib")
joblib.dump(y_scaler, f"../exports/y_scaler_{timestamp}.joblib")

['../exports/y_scaler_20260128_1404.joblib']

In [36]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Analysis.ipynb
	modified:   preprocess.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
